In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
def generate_data(num_samples):
    X = np.random.randint(0, 10, (num_samples, 5))
    y = np.flip(X, axis = 1)
    return X, y

In [3]:
num_samples = 20000
X, y = generate_data(num_samples)

In [4]:
X = X.reshape((X.shape[0], X.shape[1], 1))
y = y.reshape((y.shape[0], y.shape[1], 1))

In [6]:
model = Sequential()
model.add(LSTM(128, input_shape = (5, 1)))
model.add(Dropout(0.2))
model.add(RepeatVector(5))
model.add(LSTM(128, return_sequences = True))
model.add(Dropout(0.2))
model.add(TimeDistributed(Dense(10, activation = 'softmax')))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [9]:
early_stoping = EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)

In [10]:
model.fit(X, y, epochs = 50, validation_split = 0.2, callbacks = [early_stoping])

Epoch 1/50
500/500 [==============================] - 10s 12ms/step - loss: 2.0006 - accuracy: 0.2207 - val_loss: 1.6986 - val_accuracy: 0.3139
Epoch 2/50
500/500 [==============================] - 5s 11ms/step - loss: 1.3896 - accuracy: 0.4360 - val_loss: 0.8768 - val_accuracy: 0.7380
Epoch 3/50
500/500 [==============================] - 6s 11ms/step - loss: 0.7476 - accuracy: 0.7194 - val_loss: 0.4554 - val_accuracy: 0.9280
Epoch 4/50
500/500 [==============================] - 5s 11ms/step - loss: 0.5238 - accuracy: 0.8159 - val_loss: 0.2759 - val_accuracy: 0.9857
Epoch 5/50
500/500 [==============================] - 5s 10ms/step - loss: 0.4184 - accuracy: 0.8550 - val_loss: 0.2130 - val_accuracy: 0.9822
Epoch 6/50
500/500 [==============================] - 6s 12ms/step - loss: 0.3610 - accuracy: 0.8766 - val_loss: 0.1693 - val_accuracy: 0.9886
Epoch 7/50
500/500 [==============================] - 5s 11ms/step - loss: 0.3193 - accuracy: 0.8902 - val_loss: 0.1491 - val_accuracy: 0.982

In [11]:
def predict_reverse(input_sequence):
    input_sequence = np.array(input_sequence).reshape((1, 5, 1))
    predicted_sequence = model.predict(input_sequence)
    return np.argmax(predicted_sequence, axis = -1).reshape((5, ))

In [13]:
while True:
    user_input = input("입력 (쉼표로 구분된 5개의 숫자, 종료하려면 'quit' 입력): ")
    if user_input.lower() == "quit":
        break
    try:
        input_sequence = list(map(int, user_input.split(',')))
    
        if len(input_sequence) != 5:
            print("5개의 숫자를 입력해주세요.")
            continue
        predicted_output = predict_reverse(input_sequence)
        print(f"입력:  {input_sequence}")
        print(f"예측된 출력: {predicted_output}")

    except ValueError:
        print("유효한 숫자를 입력해주세요.")

유효한 숫자를 입력해주세요.


In [14]:
test_X, test_y = generate_data(20)

In [15]:
correct_predictions = 0
total_predictions = len(test_X)

In [17]:
for i in range(total_predictions):
    input_sequence = test_X[i]
    true_output = test_y[i].reshape(5, )
    predicted_output = predict_reverse(input_sequence)

    if np.array_equal(predicted_output, true_output):
        correct_predictions += 1

    print(f"입력: {input_sequence.reshape(5, )}")
    print(f"예측된 출력: {predicted_output}")
    print(f"실제 출력: {true_output}")
    print(f"정확 여부: {'맞음' if np.array_equal(predicted_output, true_output) else '틀림'}")

accuracy = correct_predictions / total_predictions
print(f"총 정확도: {accuracy * 100:.2f}%")

1/1 [==============================] - 1s 750ms/step
입력: [5 1 8 2 2]
예측된 출력: [2 2 8 1 5]
실제 출력: [2 2 8 1 5]
정확 여부: 맞음
1/1 [==============================] - 0s 41ms/step
입력: [4 4 2 4 3]
예측된 출력: [3 4 2 4 4]
실제 출력: [3 4 2 4 4]
정확 여부: 맞음
1/1 [==============================] - 0s 33ms/step
입력: [4 5 4 2 3]
예측된 출력: [3 2 4 5 4]
실제 출력: [3 2 4 5 4]
정확 여부: 맞음
1/1 [==============================] - 0s 35ms/step
입력: [3 3 8 5 0]
예측된 출력: [0 5 8 3 3]
실제 출력: [0 5 8 3 3]
정확 여부: 맞음
1/1 [==============================] - 0s 35ms/step
입력: [7 9 1 0 5]
예측된 출력: [5 0 1 9 7]
실제 출력: [5 0 1 9 7]
정확 여부: 맞음
1/1 [==============================] - 0s 29ms/step
입력: [7 0 7 9 8]
예측된 출력: [8 9 7 0 7]
실제 출력: [8 9 7 0 7]
정확 여부: 맞음
1/1 [==============================] - 0s 35ms/step
입력: [4 6 5 4 7]
예측된 출력: [7 4 5 6 4]
실제 출력: [7 4 5 6 4]
정확 여부: 맞음
1/1 [==============================] - 0s 26ms/step
입력: [4 7 2 5 4]
예측된 출력: [4 5 2 7 4]
실제 출력: [4 5 2 7 4]
정확 여부: 맞음
1/1 [==============================] - 0s 34ms/step
입력: [2 1 9 